# Overwrite Content with CSV

The script utilizes <code>pandas</code> dataframes to read in [311 data from Socrata]("https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9").  Export the pandas dataframe into CSV
file, overwrite service in ArcGIS Online.

----

**Directions:** Begin by importing the standard libraries we have been working with: <code>pandas</code>, <code>getpass</code>, and <code>arcgis</code>.
* <code>pandas</code> is a common toolkit for data science. It contains <code>dataframes</code> which are flexible data structures used for data manipulation and analysis.
* <code>getpass</code> for obtaining a passcode securely.
* <code>arcgis</code> for importing gis portal libraries.

In [3]:
# Import libraries
import pandas as pd
import getpass
from arcgis.gis import GIS
from arcgis import features

**Directions:** Query the JSON data layer using <code>pd.read_json()</code>. Establish the CSV output location.

Since 311 is a large dataset, we will apply a datetime query to retrieve records from midnight January 1, 2019 to March 26, 2019 (i.e., date of the workshop) 11:59:59PM (i.e., 23:59:59). (**Note:** The commented query below it can be used to test connection to the 311 JSON data and pulls 25 records.)

311 provides both WGS and State Plane coordinates, so we will drop a number of extraneous fields from the JSON data layer, including the extra location and State Plane coordinates, and view the columns in the data frame.

In [27]:
# Query Socrata directly
df = pd.read_json("https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=created_date%20between%20%272019-01-01T00:00:00%27%20and%20%272019-03-26T23:59:59%27")
# df = pd.read_json("https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$limit=25")
# Establish csv output
csv_output = r'D:\Scratch\NYC_311.csv'

#Show Data Frame
df.head()
df = df.drop('location', 1)
df = df.drop('x_coordinate_state_plane', 1)
df = df.drop('y_coordinate_state_plane', 1)
# df = df.drop(':@computed_region_92fq_4b7q', 1)
# df = df.drop(':@computed_region_efsh_h5xi', 1)
# df = df.drop(':@computed_region_f5dn_yrer', 1)
# df = df.drop(':@computed_region_sbqj_enih', 1)
# df = df.drop(':@computed_region_yeji_bk3q', 1)
# df.rename(columns={'x_coordinate_state_plane': 'X'}, inplace=True)
# df.rename(columns={'y_coordinate_state_plane': 'Y'}, inplace=True)
df.columns

Index(['address_type', 'agency', 'agency_name', 'bbl', 'borough', 'city',
       'closed_date', 'community_board', 'complaint_type', 'created_date',
       'cross_street_1', 'cross_street_2', 'descriptor', 'due_date',
       'facility_type', 'incident_address', 'incident_zip',
       'intersection_street_1', 'intersection_street_2', 'latitude',
       'location_type', 'longitude', 'open_data_channel_type', 'park_borough',
       'park_facility_name', 'resolution_action_updated_date',
       'resolution_description', 'status', 'street_name',
       'taxi_company_borough', 'taxi_pick_up_location', 'unique_key'],
      dtype='object')

**Directions:** Export the dataframe to a CSV using the previously designated output location, then view a preview of this CSV.

In [28]:
df.to_csv(csv_output, sep=',', index=False)

In [29]:
# read the initial csv
csv1 = csv_output
cities_df_1 = pd.read_csv(csv1)
cities_df_1

,address_type,agency,agency_name,bbl,borough,city,closed_date,community_board,complaint_type,created_date,...,open_data_channel_type,park_borough,park_facility_name,resolution_action_updated_date,resolution_description,status,street_name,taxi_company_borough,taxi_pick_up_location,unique_key
0,LATLONG,DOHMH,Department of Health and Mental Hygiene,NaN,QUEENS,ELMHURST,2019-01-10T00:00:00.000,Unspecified QUEENS,Rodent,2019-01-01T00:00:00.000,...,MOBILE,QUEENS,Unspecified,2019-01-10T00:00:00.000,The Department of Health and Mental Hygiene wi...,Closed,52 AVENUE,NaN,NaN,41309122
1,LATLONG,DOHMH,Department of Health and Mental Hygiene,NaN,BRONX,BRONX,2019-01-01T00:00:00.000,Unspecified BRONX,Rodent,2019-01-01T00:00:00.000,...,MOBILE,BRONX,Unspecified,2019-01-01T00:59:56.000,The Department of Health and Mental Hygiene wi...,Closed,EAST 169 STREET,NaN,NaN,41310910
2,ADDRESS,DOHMH,Department of Health and Mental Hygiene,3.042719e+09,BROOKLYN,BROOKLYN,2019-01-01T00:00:00.000,05 BROOKLYN,Rodent,2019-01-01T00:00:00.000,...,PHONE,BROOKLYN,Unspecified,2019-01-01T16:11:05.000,The Department of Health and Mental Hygiene wi...,Closed,ELDERT LANE,NaN,NaN,41314898
3,INTERSECTION,DOHMH,Department of Health and Mental Hygiene,NaN,QUEENS,Saint Albans,2019-01-03T00:00:00.000,12 QUEENS,Rodent,2019-01-01T00:00:00.000,...,PHONE,QUEENS,Unspecified,2019-01-03T00:00:00.000,The Department of Health and Mental Hygiene wi...,Closed,NaN,NaN,NaN,41315045
4,LATLONG,DOHMH,Department of Health and Mental Hygiene,NaN,BRONX,BRONX,2019-01-02T00:00:00.000,Unspecified BRONX,Rodent,2019-01-01T00:00:00.000,...,MOBILE,BRONX,Unspecified,2019-01-02T00:00:00.000,The Department of Health and Mental Hygiene wi...,Closed,CLAFLIN AVENUE,NaN,NaN,41315071
5,ADDRESS,DOHMH,Department of Health and Mental Hygiene,4.162270e+09,QUEENS,Rockaway Park,2019-01-16T00:00:00.000,14 QUEENS,Unsanitary Pigeon Condition,2019-01-01T00:00:00.000,...,PHONE,QUEENS,Unspecified,2019-01-16T00:00:00.000,The Department of Health and Mental Hygiene wi...,Closed,BEACH 118 STREET,NaN,NaN,41315216
6,ADDRESS,DOHMH,Department of Health and Mental Hygiene,3.021030e+09,BROOKLYN,BROOKLYN,2019-01-03T00:00:00.000,02 BROOKLYN,Rodent,2019-01-01T00:00:00.000,...,PHONE,BROOKLYN,Unspecified,2019-01-03T00:00:00.000,The Department of Health and Mental Hygiene wi...,Closed,CARLTON AVENUE,NaN,NaN,41315249
7,ADDRESS,DOHMH,Department of Health and Mental Hygiene,3.017780e+09,BROOKLYN,BROOKLYN,2019-01-03T00:00:00.000,03 BROOKLYN,Unsanitary Pigeon Condition,2019-01-01T00:00:00.000,...,PHONE,BROOKLYN,Unspecified,2019-01-03T00:00:00.000,The Department of Health and Mental Hygiene wi...,Closed,BEDFORD AVENUE,NaN,NaN,41315356
8,ADDRESS,DOHMH,Department of Health and Mental Hygiene,4.048220e+09,QUEENS,Flushing,2019-01-03T00:00:00.000,07 QUEENS,Unsanitary Animal Pvt Property,2019-01-01T00:00:00.000,...,ONLINE,QUEENS,Unspecified,2019-01-03T00:00:00.000,The Department of Health and Mental Hygiene wi...,Closed,150 STREET,NaN,NaN,41315359
9,ADDRESS,DOHMH,Department of Health and Mental Hygiene,3.014910e+09,BROOKLYN,BROOKLYN,2019-01-02T00:00:00.000,03 BROOKLYN,Rodent,2019-01-01T00:00:00.000,...,ONLINE,BROOKLYN,Unspecified,2019-01-02T00:00:00.000,The Department of Health and Mental Hygiene wi...,Closed,HALSEY STREET,NaN,NaN,41315446


In [30]:
# print the number of records in this csv
cities_df_1.shape

(1000, 32)

**Directions:** Connect to the GIS portal, add the CSV as an item, and then publish as a feature service.

**Note:** When publishing as a feature serivce, you MUST specify spatial reference, including the <code>latestWkid</code> and the <code>wkid</code>. 

In [39]:
# Connect to the GIS
# password = getpass()
gis = GIS("https://www.arcgis.com",'alex.brown.localgovne')
# gis = GIS('https://neenterprise.esri.com/portal', 'abrown')

Enter password: ········


In [40]:
# add the csv as an item
title = 'NYC_311_2019_SP'
item_prop = {'title': title}
csv_item = gis.content.add(item_properties=item_prop, data=csv1)
csv_item

<Item title:"NYC_311_2019_SP" type:CSV owner:alex.brown.localgovne>

In [41]:
# publish the csv item into a feature layer
new_item = csv_item.id
feature_layer_item = gis.content.get(new_item)

# State Plane New York
pparms = {"name": 'NYC_311_2019_SP', "targetSR": {"latestWkid": 2263, "wkid":102718},
          "extent":{"xmin":923594,"ymin":134684,"xmax":1066253,"ymax":270842,
                    "spatialReference":{"wkid":102718,"latestWkid":2263}}}

feature_layer_item.publish(publish_parameters=pparms)

# By Default if you web mercator
# feature_layer_item.publish()

<Item title:"NYC_311_2019_SP" type:Feature Layer Collection owner:alex.brown.localgovne>

**Directions:** After publishing, ensure the feature service exists in the GIS portal and add it to a web map.

In [42]:
# Search by title
search = gis.content.search(query="title: NYC_311_2019_SP", item_type="Feature Service")
search

[<Item title:"NYC_311_2019_SP" type:Feature Layer Collection owner:alex.brown.localgovne>]

In [43]:
# From search list, return the index for service one is interseted in connecting to
item = search[0]
itemID = item.id
itemID

'd532a2a9093b4e19aa5cf1b3f4276c11'

In [44]:
# Establish connection to Feature Layer item
feature_layer_new = gis.content.get(itemID)


In [45]:
map1 = gis.map('New York, NY')
map1

MapView(layout=Layout(height='400px', width='100%'))

In [46]:
map1.add_layer(feature_layer_new)